In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
import pandas as pd
import numpy as np
from collections import Counter
df= pd.read_csv("../Dataset/final.csv")
df.shape
df

,agent_firm_name,case_status,class_of_admission,country_of_citizenship,employer_city,employer_decl_info_title,employer_name,employer_num_employees,employer_state,foreign_worker_info_education,foreign_worker_info_inst,foreign_worker_info_major,foreign_worker_info_state,pw_level_9089,pw_soc_title,wage_offer_from_9089
0,Reinhart Boerner Van Deuren s.c.,Certified,H-1B,INDIA,WAUKESHA,Vice President of Human Resources,GENERAC POWER SYSTEMS,1935.0,WISCONSIN,Bachelor's,UNIVERSITY OF KERALA,MECHANICAL ENGINEERING,WISCONSIN,4,Industrial Engineers,90000.0
1,Law Office of Jonathan Chin,Denied,H-1B,GERMANY,AURORA,Chief Financial Officer,"AVIDITY, LLC",4.0,COLORADO,Doctorate,CHRISTIAN ALBRECHTS UNIVERSITY OF KIEL,MOLECULAR BIOLOGY,COLORADO,1,"Biological Scientists, All Other",65000.0
2,Cumming & Partners,Certified,H-1B,CANADA,NEW YORK,Managing Partner,TALBERT & TALBERT LLC,3.0,NEW YORK,Bachelor's,THE UNIVERSITY OF WESTERN ONTARIO,MEDIA STUDIES,NEW YORK,2,Public Relations Specialists,55700.0
3,"Law Offices of Kendra S. Elliott, Esq.",Certified,H-1B,BRAZIL,GALENA PARK,Human Resources Manager,"GREEN EARTH FUELS OF HOUSTON, LLC.",36.0,TEXAS,Bachelor's,CENTRO UNIVERSITARIO DE CIDADE - UNIVERCIDADE,MARKETING,TEXAS,2,Market Research Analysts and Marketing Special...,53560.0
4,"Law Office of Thomas V. Allen, PLLC",Certified,H-1B,INDIA,FEDERAL WAY,GENERAL MANAGER- OPERATIONS & DELIVERY,APPLEXUS TECHNOLOGIES LLC,36.0,WASHINGTON,Bachelor's,MOHAMED SATHAK ENGINEERING COLLEGE (ANNA UNIVE...,ENGINEERING,NEW JERSEY,3,"Software Developers, Applications",99424.0
5,Litwin & Associates,Certified,H-1B,CHINA,MILPITAS,Chief Financial Officer,"ARRAY NETWORKS, INC.",50.0,CALIFORNIA,Master's,SYRACUSE UNIVERSITY,COMPUTER SCIENCE,CALIFORNIA,2,"Software Developers, Applications",98675.0
6,Berry Appleman & Leiden LLP,Certified,H-1B,INDIA,DALLAS,International Human Resources Representative,"HUNT CONSOLIDATED, INC.",600.0,TEXAS,Master's,THE UNIVERSITY OF TEXAS AT DALLAS,COMPUTER SCIENCE,TEXAS,2,Computer Systems Analysts,69600.0
7,Armstrong Teasdale LLP,Certified,H-1B,CHINA,CHESTERFIELD,"Executive Director, Human Resources US & Mexico",RGA REINSURANCE,920.0,MISSOURI,Master's,FINANCE,WASHINGTON UNIVERSITY IN ST. LOUIS,MISSOURI,2,Financial Analysts,75000.0
8,"Fragomen, Del Rey, Bernsen & Loewy, LLP",Certified,H-1B,GUATEMALA,GLENNVILLE,General Counsel,"BLAND FARMS, LLC",776.0,GEORGIA,Bachelor's,EARTH UNIVERISTY,AGRICULTURAL ENGINEERING,GEORGIA,1,"Farmers, Ranchers, and Other Agricultural Mana...",55000.0
9,"FRAGOMEN, DEL REY, BERNSEN & LOEWY, LLP",Certified,H-1B,CHINA,PHILADELPHIA,"IMMIGRATION ADMINISTRATION, HUMAN RESOUCES",TOWERS WATSON PENNSYLVANIA INC.,3000.0,PENNSYLVANIA,Master's,GEORGIA STATE UNIVERSITY,ACTUARIAL SCIENCE,ILLINOIS,1,Financial Analysts,97850.0


# Job Titles

## Clustering using TFIDFVectorizer vectors of words
(Does not work to our liking)

In [6]:
jobs = df['employer_decl_info_title']
len(jobs.unique())
job=list(jobs)

## Manual Handling of 'Job Titles'

Reducing the number of unique job titles drastically from 12842 to 87

In [8]:
print("Before: ",len(jobs.unique()))

Before:  12842


### Reduce the job titles string to one significant word

In [9]:
l=dict()
d1=['Specialist', 'Attorney','Coordinator','Manager','leader','Administrator','generalist','assistant','vp','Director','Representative','Analyst','Associate']
d1 = [i.lower() for i in d1]
j=[x.lower() for x in jobs.unique()]
sp=[',','.','/','-',':','$','&','–']
for i in j:
    for c in sp:
        abc=i.replace(c, ' ')
    x=abc.split(' ')
    y=set(x)
    z=set(d1)
    a = y&z
    a=list(a)
    if(len(a)==0):
        if(abc=='jen.griffin@pega.com' or i=='morris@gtrr.net'):
            l[i]=np.nan
        elif(abc=='chief executive officer'):
            l[i]='ceo'
        elif(abc=='chief technology officer'):
            l[i]='cto'
        elif('vice president' in abc):
            l[i]='vp'
        elif(abc=='chief financial'):
            l[i]='cfo'
        elif(abc=='cfo-treasurer'):
            l[i]='cfo'
        elif(len(x)>2):
            if ("of" in x[0:2]) or ('&' in x[0:2]):
                l[i]=" ".join(x[0:1])
            l[i]=" ".join(x[0:2])
        else:
            l[i]="".join(x[0])
    else:
        l[i]=a[0]
l

{'vice president of human resources': 'vp',
 'chief financial officer': 'chief financial',
 'managing partner': 'managing',
 'human resources manager': 'manager',
 'general manager- operations & delivery': 'general manager-',
 'international human resources representative': 'representative',
 'executive director, human resources us & mexico': 'executive director,',
 'general counsel': 'general',
 'immigration administration, human resouces': 'immigration administration,',
 'president': 'president',
 'director of human resources': 'director',
 'director': 'director',
 'senior legal analyst': 'analyst',
 'immigration program manager': 'manager',
 'human resources administrator': 'administrator',
 'immigration specialist - qualcomm hr': 'specialist',
 'director/immigration counsel': 'director/immigration',
 'director of employee relations/attorney': 'director',
 'human resources analyst ii': 'analyst',
 'vp': 'vp',
 'vice president': 'vp',
 'immigration specialist': 'specialist',
 'chief 

In [10]:
z = []
for key,val in l.items():
    #print(x)
    try:
        val = val.split(" ")[0]
        l[key]=val
    except:
        continue
l

{'vice president of human resources': 'vp',
 'chief financial officer': 'chief',
 'managing partner': 'managing',
 'human resources manager': 'manager',
 'general manager- operations & delivery': 'general',
 'international human resources representative': 'representative',
 'executive director, human resources us & mexico': 'executive',
 'general counsel': 'general',
 'immigration administration, human resouces': 'immigration',
 'president': 'president',
 'director of human resources': 'director',
 'director': 'director',
 'senior legal analyst': 'analyst',
 'immigration program manager': 'manager',
 'human resources administrator': 'administrator',
 'immigration specialist - qualcomm hr': 'specialist',
 'director/immigration counsel': 'director/immigration',
 'director of employee relations/attorney': 'director',
 'human resources analyst ii': 'analyst',
 'vp': 'vp',
 'vice president': 'vp',
 'immigration specialist': 'specialist',
 'chief executive officer': 'ceo',
 'managing directo

In [11]:
print(len(set(l.values())))

1204


### Find out very infrequenct job titles and add label them as 'Others'

In [12]:
jobTitles= dict(Counter(l.values()))
for key,value in jobTitles.items():
    print(key, "\t",value)

vp 	 1376
chief 	 183
managing 	 91
manager 	 1566
general 	 69
representative 	 50
executive 	 102
immigration 	 91
president 	 78
director 	 1259
analyst 	 168
administrator 	 107
specialist 	 416
director/immigration 	 1
ceo 	 23
controller 	 3
nan 	 2
attorney 	 54
hr 	 248
in-house 	 17
paralegal 	 3
vp-human 	 4
generalist 	 92
senior 	 385
team 	 10
coo 	 16
principal 	 40
headmaster 	 1
owner 	 17
coordinator 	 144
manager, 	 339
global 	 104
head 	 157
editor-in-chief 	 1
associate 	 302
director, 	 596
assistant 	 247
cio 	 2
director/regional 	 2
vie 	 1
superintendent 	 4
district 	 2
corporate 	 42
asst. 	 26
cfo 	 19
cto 	 4
associate, 	 4
phr, 	 2
sr. 	 339
human 	 142
manager-immigration 	 3
vp, 	 192
chairman 	 22
leader 	 43
vice-president 	 14
lead 	 91
consultant, 	 5
kendall@chandlersharma.com 	 1
chairman/ceo 	 1
v.p 	 4
u.s. 	 11
business 	 18
superior 	 1
director-human 	 1
president/ceo 	 1
interim 	 44
treasurer 	 3
program 	 21
legal 	 14
sphr 	 1
recruiter 	

horizon91710 	 1
globaal 	 1
yoav@tadmorlaw.com 	 1
jason.cotroneo@synchronyfinancial.com 	 1
harry@ins3.com 	 1
iss 	 1
ipma-cp, 	 1
llc 	 1
vice-president-ambulatory 	 1
tom.guerriero@omnova.com 	 1
director/human 	 1
int'l 	 4
mercy 	 1
organizational 	 1
ssic 	 1
ravi@eminenceitsol.com 	 1
hrd, 	 1
specialist--edm 	 1
product 	 1
sumeshsaxena@hotmail.com 	 1
permonlinesfdc@immigrationlaw.com 	 1
olajumoke.akinleye@teamaol.com 	 1
janet.derrico@sanofi.com 	 1
mngng 	 1
technologist 	 1
sr.advisor, 	 1
humarn 	 1
co-founder/coo 	 1
mechanical 	 1
recruit 	 1
hr/people 	 1
rabbi 	 1
carol.curran@omron.com 	 1
co-director, 	 2
cpa/ceo 	 1
administration/hr 	 1
cassiopae1 	 1
owner/physician 	 1
emily@emilylove 	 1
bankofthewest 	 1
h.r.specialist 	 1
svp/cfo 	 1
principat 	 1
sylvia.cruz@alcon.com 	 1
client 	 2
hr. 	 1
donavon.corliss@nationstarmail.com 	 1
cburkhardt@ziffservices.com 	 1
e.mazo@dglogik.com 	 1
milford 	 1
mithilesh.sharma@capgemini.com 	 1
tinamunro 	 1
hr-executive 

In [13]:
jobTitles

shortJobs = {}
sj=[]
shortJobs['others'] = 0
for a in jobTitles.keys():
    if jobTitles[a]<10:
        shortJobs['others']+=jobTitles[a]
        sj.append(a)
    else:
        shortJobs[a] = jobTitles[a]

In [14]:
len(shortJobs.keys())
shortJobs['others']

1664

In [15]:
sj

['director/immigration',
 'controller',
 nan,
 'paralegal',
 'vp-human',
 'headmaster',
 'editor-in-chief',
 'cio',
 'director/regional',
 'vie',
 'superintendent',
 'district',
 'cto',
 'associate,',
 'phr,',
 'manager-immigration',
 'consultant,',
 'kendall@chandlersharma.com',
 'chairman/ceo',
 'v.p',
 'superior',
 'director-human',
 'president/ceo',
 'treasurer',
 'sphr',
 'recruiter',
 'dca',
 'coo/cfo',
 'resource',
 'director-professional',
 'exec.',
 'vice-president,',
 'employment',
 'comptroller',
 'it',
 'staff',
 'president:',
 'manager-operations',
 'administrative',
 'mgrm',
 'sdaza@aetea.com',
 'admin',
 'labor',
 'ceo/chairman',
 'visa',
 'engineering',
 'manager-u.s.',
 'manager-hr',
 'president/owner',
 'cfo/coo',
 'accounting',
 'director/owner',
 "ass't",
 'leader,',
 'hrollins@reedgroup.com',
 'chielf',
 'professor,',
 'cathyb1212',
 'technical',
 'director/ceo',
 'immigration@capitalone.com',
 'cpa/owner',
 'dentisrt',
 'group',
 'c.e.o.',
 'cpa/president',
 'past

In [16]:
for key,value in l.items():
    if(l[key] in sj):
        l[key]="others"

In [17]:
def jobConvert(x):
    return l[x.lower()]

df['employer_decl_info_title']=df['employer_decl_info_title'].apply(jobConvert)
print("After: ",len(df['employer_decl_info_title'].unique()))

After:  87


In [18]:
df.to_csv("dweepafinal.csv")

In [19]:
len(df['employer_decl_info_title'].unique())

87

In [20]:
df['employer_decl_info_title']

0                     vp
1                  chief
2               managing
3                manager
4                general
5                  chief
6         representative
7              executive
8                general
9            immigration
10             president
11              director
12              director
13             president
14             president
15               analyst
16               manager
17         administrator
18            specialist
19            specialist
20                others
21              director
22              director
23               analyst
24                    vp
25                    vp
26               general
27            specialist
28            specialist
29                   ceo
               ...      
150743         president
150744          director
150745            others
150746                vp
150747         director,
150748                hr
150749            senior
150750           manager
150751           manager
